## Reuters Newswire Dataset 
<br>
A collection of newswire data is assembled for text classification purposes, and full description of the dataset can be found at [UCI machine learning repositoty](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection). Load data to jupyter notebook with Keras. 

In [27]:
import numpy as np
import pandas as pd
from collections import Counter

#### Load Data

In [28]:
from keras.datasets import reuters 

n = 10000  # top 10000 most common words

(train_data, train_label), (test_data, test_label) = reuters.load_data(num_words=n)
print('Number of training examples: ', train_data.shape[0])
print('Number of test examples: ', test_data.shape[0])

print('Example training data: ', train_data[0])
print('Example training data label: ', train_label[0])

Number of training examples:  8982
Number of test examples:  2246
Example training data:  [1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
Example training data label:  3


#### Decode Data to Newswire

In [29]:
def decode_newswire(example):
    """
        Args:
            List of word indices 
        Returns:
            List of words matched to given indices
    """
    word_to_index = reuters.get_word_index()
    index_to_word = {key: value for (value, key) in word_to_index.items()}
    words = [index_to_word.get(i-3, 'UNK') for i in example] #indices offset by 3
    return ' '.join(words)

In [30]:
# print one example newswire
decode_newswire(train_data[0])

'UNK UNK UNK said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [31]:
print('Number of exmples for each topic label: ', Counter(train_label))

Number of exmples for each topic label:  Counter({3: 3159, 4: 1949, 19: 549, 16: 444, 1: 432, 11: 390, 20: 269, 13: 172, 8: 139, 10: 124, 9: 101, 21: 100, 25: 92, 2: 74, 18: 66, 24: 62, 0: 55, 34: 50, 12: 49, 36: 49, 28: 48, 6: 48, 30: 45, 23: 41, 31: 39, 17: 39, 40: 36, 32: 32, 41: 30, 14: 26, 26: 24, 39: 24, 43: 21, 15: 20, 38: 19, 37: 19, 29: 19, 45: 18, 5: 17, 7: 16, 27: 15, 22: 15, 42: 13, 44: 12, 33: 11, 35: 10})


### 1. Data Preprocessing 

All observations in traning dataset are lists of word indices. 

#### Construct Vectorized Input Data 

In [32]:
def construct_input_vectors(X, N):
    """vectorize newswire data"""
    input = np.zeros((X.shape[0], N))
    for i in range(X.shape[0]):
        for j in range(len(X[i])):
            input[i][X[i][j]] = 1
    return input

In [33]:
X_train = construct_input_vectors(train_data, n)
y_train = train_label
X_test = construct_input_vectors(test_data, n)
y_test = test_label

In [34]:
#y_train = pd.get_dummies(train_label).values

### 2. Construct Neural Network 



In [35]:
def relu_activation(X, W, b):
    Z = np.maximum(np.dot(X, W) + b, 0) # element-wise max between two arrays
    return Z

In [36]:
def softmax(A):
    exps = np.exp(A - np.max(A, axis=1, keepdims=True)) # prevent overflow
    return exps / np.sum(exps, axis=1, keepdims=True) 

In [37]:
def cross_entropy_loss(model_output, target):
    ce = -np.sum(target * np.log(model_output) + (1 - target) * np.log(1 - model_output))
    return ce

In [38]:
def back_propagation(delta, X):  
    gradient = np.dot(X.T, delta)
    gradient[X < 0] = 0
    return gradient

In [39]:
def evaluate_accuracy(y, X, W1, b1, W2, b2):
    A1 = relu_activation(X, W1, b1)
    class_prob = np.dot(A1, W2) + b2
    pred = np.argmax(class_prob, axis=1)
    print('prediction accuracy: %.2f%%' % (100 * np.mean(pred == y)))

In [40]:
h = 100 # size of hidden layer
num_classes = 46 # number of classes
batch_size = X_train.shape[0]
learning_rate = 1
epochs = 100

In [41]:
# initialize parameters 
np.random.seed(0)

W1 = 0.01 * np.random.randn(n, h)
b1 = np.zeros((1, h))
W2 = 0.01 * np.random.randn(h, num_classes)
b2 = np.zeros((1, num_classes))

In [42]:
# batch gradient descent 

for i in range(epochs):
    
    # forward propogation
    A1 = relu_activation(X_train, W1, b1) 
    A2 = np.dot(A1, W2) + b2
    probs = softmax(A2)  
    
    # compute cross entropy loss 
    logprob = -np.log(probs[range(batch_size), y_train])
    loss = np.sum(logprob) / batch_size
       
    # compute the gradient on scores
    dscores = probs
    dscores[range(batch_size), y_train] -= 1
    dscores /= batch_size
      
    # backprop W2 b2
    dW2 = np.dot(A1.T, dscores)
    db2 = np.sum(dscores, axis=0, keepdims=True)
    
    # backprop into hidden layer
    dhidden = np.dot(dscores, W2.T)
    dhidden[A1 <= 0] = 0
    
    # backprop W1 b1
    dW1 = np.dot(X_train.T, dhidden)
    db1 = np.sum(dhidden, axis=0, keepdims=True)
  
    # update weights
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    if i % 10 == 0:
        print("epoch {0}: loss {1}".format(i, loss)) 
        # evaluate training set accuracy
        evaluate_accuracy(y_train, X_train, W1, b1, W2, b2)

epoch 0: loss 3.8271235620616832
prediction accuracy: 35.17%
epoch 10: loss 2.305714185084428
prediction accuracy: 46.79%
epoch 20: loss 1.8440828770181341
prediction accuracy: 57.48%
epoch 30: loss 1.5867835288424288
prediction accuracy: 64.89%
epoch 40: loss 1.4334826360898312
prediction accuracy: 69.41%
epoch 50: loss 1.2946896739726377
prediction accuracy: 70.74%
epoch 60: loss 1.2125398928937663
prediction accuracy: 71.67%
epoch 70: loss 1.1571459581867884
prediction accuracy: 72.66%
epoch 80: loss 1.0795161375162003
prediction accuracy: 74.60%
epoch 90: loss 1.0348482956113996
prediction accuracy: 75.54%


In [43]:
# evaluate test set accuracy
evaluate_accuracy(y_test, X_test, W1, b1, W2, b2)

prediction accuracy: 71.37%
